In [ ]:
镜像构建和加载过程中出现的问题与解决办法

In [ ]:
1. dockerfile 编写

In [ ]:
1.1 首先是基础环境
    一开始使用了cuda版本的ubuntu20.04, 出现了无法链接到镜像网站的情况，后来使用了 nvcr.io 镜像网站上的系统，成功编译；
    但是编译成功后，整个镜像达到了44G，而且有几个重要的结论：
    （1）基础环境的选择
    “如果目标对象已经安装了ubuntu和cuda，那么不需要构建系统层面的基础环境，可以直接使用python环境”
    “python 环境依然使用了镜像网站”：
    FROM m.daocloud.io/docker.io/library/python:3.8.19
    （2）环境报错处理
    基础环境报错基本上都是网络原因，针对使用的基础环境，搜索可靠的镜像网站即可。 

In [ ]:
1.1.1 基础环境选择的原则
如果是针对目标宿主机进行镜像迁移，需要根据宿主机的状态选择基础镜像，避免镜像过大，启动和编译时间很长，基本逻辑如下：
step1. 确认宿主机的基本状况
    操作系统：确认宿主机的操作系统类型（如 Ubuntu 20.04）。
    GPU 和 CUDA 配置：是否安装了 NVIDIA GPU 驱动？
    CUDA toolkit 的版本（如 12.4）是否已安装？
    是否安装了 cuDNN 等相关库？
    可用资源：CPU、内存和显存的使用情况。
    是否安装了 NVIDIA Container Toolkit（包括 nvidia-docker2）？

In [ ]:
step1.5 NVIDIA Container Toolkit 安装
    一般情况下，ubuntu 系统安装后都会配置 CUDA 和 GPU 驱动，随之，cuDNN 等库也会安装。但是一般情况下 NVIDIA CONTAINER TOOLKIT 都不会安装。
    （1）NVIDIA Container Toolkit（包括 nvidia-docker2）应当安装在宿主机上。
        用来让 Docker 引擎识别和管理 NVIDIA GPU 资源的工具，从而能在容器中调用宿主机的GPU，因此必须安装在运行 Docker 的机器上。
    （2）安装步骤（以 ubuntu 20.04 为例）
        # 更新包列表并安装必要的依赖工具
        sudo apt-get update && sudo apt-get install -y curl gnupg2 software-properties-common
        # 添加 NVIDIA 官方仓库
        distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
        curl -s -L https://nvidia.github.io/libnvidia-container/gpgkey | sudo apt-key add -
        curl -s -L https://nvidia.github.io/libnvidia-container/$distribution/libnvidia-container.list | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list
        # 更新软件源
        sudo apt-get update
        # 安装 NVIDIA Container Toolkit
        sudo apt-get install -y nvidia-container-toolkit
        # 重启docker服务
        sudo systemctl restart docker
        # 验证安装，如果有GPU信息输出，则安装成功
        sudo docker run --rm --gpus all nvidia/cuda:11.0-base nvidia-smi
    （3）常见问题
        # 权限问题，如果遇到权限错误，将用户加入docker组
        sudo usermod -aG docker $USER
        newgrp docker
        # 或者直接进入root
        sudo su -
        # 如果通过代理访问网络，需要在 Docker 中配置代理
        sudo mkdir -p /etc/systemd/system/docker.service.d
        sudo tee /etc/systemd/system/docker.service.d/http-proxy.conf <<EOF
        [Service]
        Environment="HTTP_PROXY=http://proxy.example.com:8080"
        Environment="HTTPS_PROXY=http://proxy.example.com:8080"
        EOF
        sudo systemctl daemon-reload
        sudo systemctl restart docker
        # 其他操作系统
        CENTOS/RHEL ： 步骤类似，需使用`yum`代替`apt-get`。
        Windows:需通过Docker Desktop的NVIDIA GPU支持功能启用。

In [ ]:
step2. 基础环境的选择
    # 直接使用python官方镜像
    如果宿主机已经完成 CUDA GPU驱动 NVDIA Container Toolkit，可以直接使用 python 官方镜像来运行基于GPU的深度学习任务
    FROM python:3.8 # 指令内容全部小写
    # 如果需要更灵活的配置，比如额外安装依赖或系统工具，可以选择基于 ubuntu 的基础镜像
    FROM ubuntu：20.04
    如果宿主机已经安装了cuda，可以在容器内使用 nvidia-cuda-toolkit 等库，不需要在容器内安装cuda
    # 使用预构建的深度学习镜像：
    FROM nvidia/cuda:12.4-base-ubuntu20.04
    预构建的镜像，需要安装python
    RUN apt-get update && apt-get install -y python3.8 python3-pip
    # 依赖管理：
    在 Dockerfile 中避免冗余安装。如果宿主机已经提供了某些库（如 CUDA），不需要在容器中重新安装。
    确保镜像中的软件版本与宿主机的环境兼容

In [ ]:
step3. 下载基础镜像
    基础镜像一般需要通过镜像网站下载，比如 m.daoicloud.io，nvcr.io 等，这个需要自己搜；
    编译基础镜像时，会出现很多 SHA，不用管，这是正常的逐层构建，最后会生成一个。

In [ ]:
1.2 LABEL
    MAINTAINER 指令已经不被最新版的 docker 支持，只能使用 LABEL；
    LABEL 需要输入两个参数：
    LABEL creater="Chengleng_Han"

In [ ]:
1.3 替换软件源：
    RUN echo "deb https://mirrors.tuna.tsinghua.edu.cn/debian/ bookworm main contrib non-free" > /etc/apt/sources.list && \
    echo "deb https://mirrors.tuna.tsinghua.edu.cn/debian/ bookworm-updates main contrib non-free" >> /etc/apt/sources.list && \
    echo "deb https://mirrors.tuna.tsinghua.edu.cn/debian/ bookworm-backports main contrib non-free" >> /etc/apt/sources.list && \
    echo "deb https://mirrors.tuna.tsinghua.edu.cn/debian-security/ bookworm-security main contrib non-free" >> /etc/apt/sources.list

In [ ]:
1.4 安装系统依赖项，并移除碎片
    RUN apt-get update && apt-get install -y build-essential && rm -rf /var/lib/apt/lists/*
    RUN apt-get clean

In [ ]:
1.5 WORKDIR
    我将WORKDIR直接设置为容器的根目录，而且我发现，虽然我使用的时Python官方环境作为基础环境，但是依然能在容器中看到linux系统的影子；
    WORKDIR /root

In [ ]:
1.6 拷贝代码
    代码的拷贝要根据项目的要求来，我使用了 wheel_legged_gym 作为尝试，安装要求是：
    （1）python 3.8
    （2）安装 pytorch （我使用了12.1 版本的官方链接）
    （3）安装 isaac gym，这个需要复制文件并 pip install -e . 安装
    （4）安装 wheel_legged_gym，也是复制文件并 pip install -e . 安装

In [ ]:
1.6.1 文件准备
    （1）在项目文件的根目录下 touch dockerfile，然后使用记事本编写（我不喜欢vim）
    （2）生成 requirements.txt
        （2.1）我使用了 pipreqs 生成 requirements.txt 文件
        pip install pipreqs
        # 在根目录下创建（随便一个位置，后面可以移动到根目录）
        pipreqs ./
        （2.2）因为我是调试成功后构建的，在我安装过程中，装了pytorch之后只安装了 dill,imaegio 并修正了 pandas 版本
        所以，我在生成的 txt 文件中只保留了这三个，并利用 conda list 限制了三个库的具体版本
1.6.2 拷贝文件
    COPY . /root

In [ ]:
1.7 RUN命令安装
    这里我是根据调试时使用的指令顺序依次写的命令：安装pytorch,安装requirements.txt(对pytorch安装后的依赖库进行修正)，本地安装isaacgym，本地安装wheel
    RUN pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
    RUN pip install --no-cache-dir -r requirements.txt # --no-cache-dir 不保留缓存

    RUN cd /root/isaacgym/python && pip install -e . \
     && cd /root && pip install -e .

In [ ]:
1.8 EXPOSE 端口（后面好像没用到）
    EXPOSE 22
    EXPOSE 6006
    EXPOSE 8888

In [ ]:
1.9 CMD命令
    我一开始想在启动容器后直接运行:
    CMD ["python", "wheel_legged_gym/scripts/train.py","--task=wheel_legged_vmc_flat"]
    但是这样会存在一个隐患：一旦报错，会直接退出容器，导致无法进行 exec
    推荐 CMD ["/bin/bash"]
    进入容器后，执行空命令行，然后通过命令行指令，找到路径（进入的是root），在执行python程序，能保证不错。

1.9.1 报错处理：如果报了段错误 Segmentation fault(core dumped)
    (1) 首先启用无渲染
        --headless，任何强化学习训练都需要 --headless
    (2) 如果还报错，一般就是容器内的驱动出了问题：
        在容器内运行nvcc --version时出现“command not found”错误。这说明容器内没有安装
        CUDA工具包，或者CUDA的路径没有正确配置。
        # 下面是在容器内操作
# 在容器内安装与 PyTorch 匹配的 CUDA 版本（PyTorch 2.4.0+cu121 对应 CUDA 12.1）：
    # 更新包管理器
    apt-get update
    # 安装依赖项
    apt-get install -y wget build-essential
    # 下载 CUDA 12.1 安装包
    wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
    # 安装 CUDA（选择默认选项）
    sh cuda_12.1.0_530.30.02_linux.run --toolkit --silent --override
    # 配置环境变量
    echo 'export PATH=/usr/local/cuda-12.1/bin:$PATH' >> ~/.bashrc
    echo 'export LD_LIBRARY_PATH=/usr/local/cuda-12.1/lib64:$LD_LIBRARY_PATH' >> ~/.bashrc
    source ~/.bashrc
    # 验证CUDA版本
    nvcc --version
    # 检查 CUDA 库路径
    ls /usr/local/cuda-12.1/lib64/libcudart.so*
    # 清除旧的编译缓存并重新触发扩展编译
    rm -rf /root/.cache/torch_extensions/py38_cu121/gymtorch
    python train.py --task=wheel_legged_vmc_flat

In [ ]:
1.9.2 报错后容器内关键检查点
 **CUDA 与 PyTorch 版本匹配**  
   确保容器内的 CUDA 版本与 PyTorch 的 `cu121` 后缀一致：
   python -c "import torch; print(torch.__version__)"
 **环境变量配置**  
   确认 `PATH` 和 `LD_LIBRARY_PATH` 包含 CUDA 路径：
   echo $PATH
   echo $LD_LIBRARY_PATH
 **GPU 驱动兼容性**  
   宿主机 NVIDIA 驱动版本需兼容 CUDA 12.1（建议驱动版本 ≥ 525.60.13）：
   nvidia-smi  # 在宿主机运行

In [ ]:
2. 镜像保存
在build完成之后，docker save -o bipeda.tar bipeda 生成一个tar文件，注意：tar文件的名字一定要和镜像名字一致，我就少输入了一个l，找了半天原因。。。

In [ ]:
3. 加载镜像
（1）传输镜像文件(tar)，复制到目标电脑
（2）加载镜像(镜像名称和标签会保留原始信息，可以通过docker images查看)：
    # 在目标电脑上执行
    docker load -i bipeda.tar
    # 或
    docker load < bipeda.tar

In [ ]:
4. 运行镜像
（1）启动容器
docker run -itd --name container_name <镜像名>:<标签>
# -d：后台运行，-i：交互式运行，-t:创建一个tty伪终端，-p：宿主机端口：容器端口，-v:宿主机目录：容器目录
（2）验证容器运行
docker ps # 查看运行中的容器

In [ ]:
5. 进入容器操作
（1）进入容器终端
docker exec -it my_container /bin/bash
# 使用 exit 退出容器终端（容器不会停止运行）
（2）容器内操作技巧
安装软件：apt-get update && apt-get install vim
修改配置文件：vim /path/to/config
其他操作与普通Linux环境一致

In [ ]:
6. 镜像内容更新
方法1：通过容器提交新镜像
（1）停止容器（可选）
docker stop my_container
（2）提交修改
docker commit -a "作者名" -m "修改描述" my_container 新镜像名：新标签
# docker commit -a "Chengleng_han" -m "Added vim" my_container myapp:v2
 (3) 验证新镜像
docker images # 查看新生成的镜像

In [ ]:
方法2：通过 Dockerfile 重建镜像（推荐）
（1）保存容器修改记录-在容器内操作后，将修改步骤写入Dockerfile:
FROM 原镜像名：标签
RUN apt-get update && apt-get install -y vim
（2）构建新镜像
docker build -t 新镜像名：新标签 .

In [ ]:
7. 其他常用命令与注意事项：
（1）删除旧镜像
docker rmi <镜像ID>
（2）导出更新后的镜像
docker save 新镜像名：标签 > updated_image.tar
（3）镜像与容器关系
直接修改容器不会影响原始镜像，必须通过 docker commit 或重建镜像持久化变更。
（4）数据持久化
重要数据建议通过 -v 挂载卷，避免容器删除后丢失。
（5）生产环境建议
优先使用 Dockerfile 构建镜像，而非手动 commit，确保可重复性和版本控制。